In [1]:
from elasticsearch import Elasticsearch

In [2]:
es = Elasticsearch(
    "https://localhost:9200",
    basic_auth=("elastic","QGm3UkTIuPLi*PAdmWjR"),
    ca_certs="C:/Users/Geraldus Wilsen/Downloads/elasticsearch-8.11.1/config/certs/http_ca.crt"
)
es.ping()

False

In [3]:
import pandas as pd

df = pd.read_csv("Medicine_Details.csv").loc[:499]
df.fillna("None", inplace=True)
df.head()

,Medicine Name,Composition,Uses,Side_effects,Image URL,Manufacturer,Excellent Review %,Average Review %,Poor Review %
0,Avastin 400mg Injection,Bevacizumab (400mg),Cancer of colon and rectum Non-small cell lun...,Rectal bleeding Taste change Headache Noseblee...,"https://onemg.gumlet.io/l_watermark_346,w_480,...",Roche Products India Pvt Ltd,22,56,22
1,Augmentin 625 Duo Tablet,Amoxycillin (500mg) + Clavulanic Acid (125mg),Treatment of Bacterial infections,Vomiting Nausea Diarrhea Mucocutaneous candidi...,"https://onemg.gumlet.io/l_watermark_346,w_480,...",Glaxo SmithKline Pharmaceuticals Ltd,47,35,18
2,Azithral 500 Tablet,Azithromycin (500mg),Treatment of Bacterial infections,Nausea Abdominal pain Diarrhea,"https://onemg.gumlet.io/l_watermark_346,w_480,...",Alembic Pharmaceuticals Ltd,39,40,21
3,Ascoril LS Syrup,Ambroxol (30mg/5ml) + Levosalbutamol (1mg/5ml)...,Treatment of Cough with mucus,Nausea Vomiting Diarrhea Upset stomach Stomach...,"https://onemg.gumlet.io/l_watermark_346,w_480,...",Glenmark Pharmaceuticals Ltd,24,41,35
4,Aciloc 150 Tablet,Ranitidine (150mg),Treatment of Gastroesophageal reflux disease (...,Headache Diarrhea Gastrointestinal disturbance,"https://onemg.gumlet.io/l_watermark_346,w_480,...",Cadila Pharmaceuticals Ltd,34,37,29


In [4]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-mpnet-base-v2')

c:\Users\Geraldus Wilsen\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
es.ping()

False

In [6]:
df["uses_vector"] = df["Uses"].apply(lambda x: model.encode(x))

In [7]:
df['id'] = range(1, len(df) + 1)
df = df[['id','Medicine Name','Uses','uses_vector']]
df.rename(columns={'Medicine Name': 'name', 'Uses':'uses'}, inplace=True)
df.head()

,id,name,uses,uses_vector
0,1,Avastin 400mg Injection,Cancer of colon and rectum Non-small cell lun...,"[0.026428059, 0.04972668, -0.03242681, -0.0772..."
1,2,Augmentin 625 Duo Tablet,Treatment of Bacterial infections,"[0.04261984, 0.0034055815, -0.004333484, -0.03..."
2,3,Azithral 500 Tablet,Treatment of Bacterial infections,"[0.04261984, 0.0034055815, -0.004333484, -0.03..."
3,4,Ascoril LS Syrup,Treatment of Cough with mucus,"[0.0103752995, 0.02946366, 0.0053328695, 0.014..."
4,5,Aciloc 150 Tablet,Treatment of Gastroesophageal reflux disease (...,"[0.06715654, -0.048785236, 0.0017081047, -0.02..."


In [8]:
len(df['uses_vector'][0])

768

In [9]:
from indexMapping import indexMapping

es.indices.create(index="medicine_data", mappings=indexMapping)

BadRequestError: BadRequestError(400, 'resource_already_exists_exception', 'index [medicine_data/SmPyFiLvSYydmoVF-NqnGA] already exists')

In [ ]:
record_list = df.to_dict("records")

In [ ]:
for record in record_list:
    try:
        es.index(index="medicine_data", document=record, id=record["id"])
    except Exception as e:
        print(e)

In [ ]:
es.count(index="medicine_data")

ObjectApiResponse({'count': 500, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}})

In [11]:
input_keyword = "i'm looking for cancer medicine"
vector_of_input_keyword = model.encode(input_keyword)

query = {
    "field" : "uses_vector",
    "query_vector" : vector_of_input_keyword,
    "k" : 10,
    "num_candidates" : 500, 
}

res = es.knn_search(index="medicine_data", knn=query , source=["name","uses"])
res["hits"]["hits"]

C:\Users\Geraldus Wilsen\AppData\Local\Temp\ipykernel_15480\3605240143.py:11: ElasticsearchWarning: The kNN search API has been replaced by the `knn` option in the search API.
  res = es.knn_search(index="medicine_data", knn=query , source=["name","uses"])


[{'_index': 'medicine_data',
  '_id': '1',
  '_score': 0.5046534,
  '_source': {'name': 'Avastin 400mg Injection',
   'uses': ' Cancer of colon and rectum Non-small cell lung cancer Kidney cancer Brain tumor Ovarian cancer Cervical cancer'}},
 {'_index': 'medicine_data',
  '_id': '16',
  '_score': 0.4488577,
  '_source': {'name': 'Armotraz Tablet', 'uses': ' Breast cancer'}},
 {'_index': 'medicine_data',
  '_id': '253',
  '_score': 0.4488577,
  '_source': {'name': 'Altraz Tablet', 'uses': ' Breast cancer'}},
 {'_index': 'medicine_data',
  '_id': '286',
  '_score': 0.42753205,
  '_source': {'name': 'Aprecap 125/80 Capsule',
   'uses': ' Nausea and vomiting due to chemotherapy'}},
 {'_index': 'medicine_data',
  '_id': '46',
  '_score': 0.42140177,
  '_source': {'name': 'Augmentin DDS Suspension',
   'uses': 'Treatment of Resistant Tuberculosis (TB)Treatment of Bacterial infections'}},
 {'_index': 'medicine_data',
  '_id': '132',
  '_score': 0.42140177,
  '_source': {'name': 'Advent Forte